Логистическая регрессия. Log Loss
1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log (как вариант - использовать np.clip или np.where).
2. На данных из урока изучите влияние гиперпараметров на ошибку алгоритма.
3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса "1". На вход функции подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model.
4. Создайте функцию calc_pred, возвращающую предсказанные классы (0 или 1). На вход функции подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model, а также порог вероятности.

In [15]:
import numpy as np
import matplotlib.pyplot as plt

In [16]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

In [17]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [18]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

In [22]:
def calc_logloss(y, y_pred):
    y_pred = np.clip(y_pred, 0.001, 0.999)
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [23]:
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.2])
calc_logloss(y1, y_pred1)

0.11207202582389662

2. На данных из урока изучите влияние гиперпараметров на ошибку алгоритма.

In [24]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [25]:
z = np.linspace(-10, 10, 101)

In [26]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [37]:
W = eval_model(X_st, y, iterations=600, eta=1e-1)

0 [ 0.45877546 -0.2833519   0.6449505   1.46641523] 1.1785958344356262
60 [-0.0128424  -0.72688689  0.89143875  1.3544156 ] 0.48721469994554284
120 [-0.24072722 -0.77003525  0.97127317  1.52658787] 0.4719426428429781
180 [-0.43352513 -0.81071331  0.98851886  1.69748012] 0.46050085078302827
240 [-0.61543205 -0.84254915  0.97970004  1.85235904] 0.4507791467362616
300 [-0.79114327 -0.86672849  0.9559449   1.99242216] 0.44215550950847404
360 [-0.96174463 -0.88506413  0.92235774  2.12056499] 0.4343123411025503
420 [-1.12761197 -0.89902764  0.88202737  2.23923918] 0.4270675837765757
480 [-1.28896325 -0.90972179  0.83707147  2.35030987] 0.42030973218301526
540 [-1.44597215 -0.91796771  0.78901738  2.45518243] 0.41396619525012956


Увеличение итераций сильно не влияет на скорость уменьшения ошибки. Уменьшение шага хорошо повлияло на скорость уменьшения ошибки.

3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса "1". На вход функции подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model.

In [46]:
def calc_pred_proba(X, W):
    y_pred_proba = sigmoid(np.dot(X, W))
    return y_pred_proba

In [48]:
y_pred_proba = calc_pred_proba(X_st, W)
y_pred_proba

array([0.33358741, 0.40447536, 0.7889237 , 0.014251  , 0.81186825,
       0.66596935, 0.90406329, 0.16584816, 0.31684868, 0.84552038])

4. Создайте функцию calc_pred, возвращающую предсказанные классы (0 или 1). На вход функции подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model, а также порог вероятности.

In [49]:
def calc_pred(X, W, threshold):
    m = X.shape[0]
    
    y_predicted = np.zeros(m)

    y_pred_proba = sigmoid(np.dot(X, W))
    
    for i in range(X.shape[0]):
        if (y_pred_proba[i] > threshold):
            y_predicted[i] = 1
        elif (y_pred_proba[i] <= threshold):
            y_predicted[i] = 0
            
    return y_predicted

In [56]:
y_predicted = calc_pred(X_st, W, 0.5)
y_predicted

array([0., 0., 1., 0., 1., 1., 1., 0., 0., 1.])

In [57]:
y_predicted = calc_pred(X_st, W, 0.9)
y_predicted

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])